### 1. Функция для фильтрации базы данных PDB.

#### 1. Условия выдачи true на выходе функции: (всегда проверяем model[0])

1) присутствует 1 цепочка (chain)

2) только $\alpha$-аминокислоты (везде присутствуют $C_{\alpha}$)

3) вода (HOH) не находится посередине белка

4) во всех residue есть все атомы (N, $C_{\alpha}$, C, атомы радикала - проверяем из json файла)

5) нет пропусков аминокислот

In [1]:
def check_function(struct):
    check_model = list(struct.get_models())[0]
    keys_arr = np.array(list(data.keys()))
    if len(list(check_model.get_chains())) != 1:
        return False
    arr_res = np.array([el.get_resname() for el in check_model.get_residues()])
    arr_res = arr_res[arr_res != 'HOH']
    if arr_res.size < 100:
        return False
    i = list(check_model.get_residues())[0].get_id()[1]
    met_hoh = 0
    for name in list(check_model.get_residues()):
        if name.get_id()[1]  != i:
            return False
        i += 1
        if not(name.get_resname() in keys_arr):
            return False
        if name.get_resname() == 'HOH':
            met_hoh = 1
            continue
        if met_hoh == 1:
            return False
        arr_atoms = [el.get_name() for el in name.get_atoms()]
        need_atoms = np.array(data[name.get_resname()]).reshape(-1)
        need_atoms = np.append(need_atoms, 'N')
        need_atoms = np.append(need_atoms, 'C')
        need_atoms = np.append(need_atoms, 'CA')
        transformed_arr = np.unique(need_atoms)
        diff = np.setdiff1d(transformed_arr, np.array(arr_atoms))
        if diff.size != 0:
            return False

    return True

